# Projet : Détection de constructions illégales

## Introduction

### Mise en place de l'environnement de travail

#### Récupération des fichiers

##### Cadastre

https://geoservices.ign.fr/bdtopo

1) Télécharger les fichiers du département désiré en faisant attention à l'année
2) Déziper les fichiers dans un répertoire *Cadastre* au même niveau que ce Jupyterbook

##### Images satellite

https://geoservices.ign.fr/bdortho

1) Télécharger les fichiers du département désiré en faisant attention à l'année
2) Déziper les fichiers dans un répertoire *Satellite* au même niveau que ce Jupyterbook

#### Bibliothèques nécessaires

**rasterio** - Bibliothèque pour la manipulation d'images raster (images géospatiales) :

*pip install rasterio*

**matplotlib** - Bibliothèque pour la création de visualisations graphiques :

*pip install matplotlib*

**numpy** - Bibliothèque pour la manipulation de tableaux multidimensionnels :

*pip install numpy*

**pandas** - Bibliothèque pour la manipulation, l'analyse et la visualisation de données de tableaux multidimensionnels :

*pip install pandas*

**geopandas** - Extension de la bibliothèque pandas pour la manipulation de données géospatiales (elle nécessite quelques dépendances) :

*pip install geopandas fiona geopy pyproj shapely*

## 0) Bibliothèques et fonctions

### Bibliothèques

#### Import des bibliothèques

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import folium
import geopandas as gpd
import glob
import plotly.express as px
import plotly.graph_objs as go
import cv2



### Fonctions

#### Fonction de récupération des images satellites

In [ ]:
def getImg(path):
    """
    Récupère de manière récursive les chemins d'image et de fichiers .tab dans un répertoire donné.

    Args:
        directory (str): Le chemin absolu du répertoire à explorer.

    Returns:
        Un dataframe avec deux colonnes :
            - La première colonne contient les chemins d'image trouvés.
            - La deuxième colonne contient les chemins de fichiers .tab trouvés.
    """
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', ".jp2"]  # Extensions d'image supportées
    tab_extensions = ['.tab']  # Extensions de fichier .tab supportées
    image_paths = []
    tab_paths = []

    for dirpath, _, filenames in os.walk(path):
        for filename in filenames:
            if any(filename.lower().endswith(ext) for ext in image_extensions):
                image_path = os.path.join(dirpath, filename)
                image_paths.append(image_path)
            elif any(filename.lower().endswith(tab) for tab in tab_extensions):
                tab_path = os.path.join(dirpath, filename)
                tab_paths.append(tab_path)

    # Créer un dictionnaire à partir des deux listes
    data = {'Image Paths': image_paths, 'Tab Paths': tab_paths}

    # Créer un DataFrame à partir du dictionnaire
    df = pd.DataFrame(data)

    return df

#### Fonction de nettoyage de répertoire

In [ ]:
def emptyPath(path):
    # Vérifier si le répertoire existe
    if os.path.exists(path):
        # Parcourir tous les fichiers et dossiers dans le répertoire
        for element in os.listdir(path):
            element_path = os.path.join(path, element)  # Chemin complet de l'élément
            if os.path.isfile(element_path):  # Vérifier si l'élément est un fichier
                os.remove(element_path)  # Supprimer le fichier
            elif os.path.isdir(element_path):  # Vérifier si l'élément est un dossier
                os.rmdir(element_path)  # Supprimer le dossier vide


#### Fonction de découpe de l'image

In [ ]:
def splitImg(nbPxX, nbPxY, pathImg, pathDest):
    """
    Découpe une image en sous-images de taille donnée et les enregistre dans un répertoire de destination.

    Args:
        nbPxX (int): Nombre de pixels en largeur pour chaque sous-image.
        nbPxY (int): Nombre de pixels en hauteur pour chaque sous-image.
        pathImg (str): Chemin absolu de l'image principale à découper.
        pathDest (str): Chemin absolu du répertoire de destination pour les sous-images.
    """
    # Vérifier si le répertoire existe déjà
    if not os.path.exists(pathDest):
        # Créer le répertoire s'il n'existe pas
        os.makedirs(pathDest)

    # Récupère la latitude et la longitude depuis le nom de l'image
    # Ouvrir l'image principale avec rasterio
    with rasterio.open(pathImg) as src:
        # Lire les propriétés de l'image principale
        width = src.width
        height = src.height
        count = src.count
        dtype = src.dtypes[0]

        # Calculer le nombre de sous-images en largeur et en hauteur
        num_subimages_x = width // nbPxX
        num_subimages_y = height // nbPxY
        
        # Obtenir le nom de l'image original
        nomImage = pathImg.split("/")[-1].split(".")[0]
        
        # Boucle pour découper les sous-images
        for i in range(num_subimages_y):
            for j in range(num_subimages_x):
                # Lire les pixels de la sous-image à partir de l'image principale
                window = rasterio.windows.Window(j * nbPxX, i * nbPxY, nbPxX, nbPxY)
                subimage = src.read(window=window)

                # Créer un chemin de fichier pour la sous-image en utilisant les coordonnées de la fenêtre
                subimage_filename = nomImage + '_' + str(i).zfill(3) + '_' + str(j).zfill(3) + '.png'  # Changer l'extension de fichier si nécessaire
                subimage_filepath = os.path.join(pathDest, subimage_filename)

                # Vérifier si la sous-image déborde de l'image principale et la compléter avec des pixels blancs si nécessaire
                if i * nbPxY + nbPxY > height:
                    subimage = np.pad(subimage, ((0, i * nbPxY + nbPxY - height), (0, 0), (0, 0)), mode='constant', constant_values=255)
                if j * nbPxX + nbPxX > width:
                    subimage = np.pad(subimage, ((0, 0), (0, j * nbPxX + nbPxX - width), (0, 0)), mode='constant', constant_values=255)

                # Enregistrer la sous-image dans le répertoire de destination
                with rasterio.open(subimage_filepath, 'w', driver='PNG', width=nbPxX, height=nbPxY, count=count, dtype=dtype) as dst:
                    dst.write(subimage)

#### Fonction de normalisation

In [ ]:
def normalize_band(band, lower_percentile=2, upper_percentile=98):
    lower_value, upper_value = np.percentile(band, (lower_percentile, upper_percentile))
    return np.clip((band - lower_value) / (upper_value - lower_value), 0, 1)

#### Fonction d'affichage

In [ ]:
def displayImg(path):
    # Read the individual red, green, and blue bands using Rasterio
    with rasterio.open(path, 'r') as src:
        red_band = src.read(1).astype(np.float32)

    with rasterio.open(path, 'r') as src:
        green_band = src.read(2).astype(np.float32)

    with rasterio.open(path, 'r') as src:
        blue_band = src.read(3).astype(np.float32)

    # Normalize the bands
    normalized_red_band = normalize_band(red_band)
    normalized_green_band = normalize_band(green_band)
    normalized_blue_band = normalize_band(blue_band)

    # Stack the bands together
    rgb_image = np.stack((normalized_red_band, normalized_green_band, normalized_blue_band), axis=-1)

    return rgb_image

#### Fonction d'affichage de plusieurs Images

In [ ]:
def displayMultipleImg(path, nbCol, nbLines, displayLegend = False):
    # Créer une figure avec des sous-graphiques
    fig, axes = plt.subplots(nbLines, nbCol, figsize=(100, 100))
    
    # Parcourir les fichiers dans le répertoire
    for i, fichier in enumerate(os.listdir(path)):
        if fichier.endswith(".png"):  # Vérifier que le fichier est une image png
            chemin_image = os.path.join(path, fichier)
            image = plt.imread(chemin_image)

            # # Lire la propriété bound de l'image
            # with rasterio.open(chemin_image) as src:
            #     xmin, ymin, xmax, ymax = src.bounds

            # Récupérer l'axe correspondant au sous-graphique courant
            axe = axes.flat[i]
            
            # Afficher l'image sur l'axe
            axe.imshow(image)
            
            # Récupérer la largeur et la hauteur de l'image
            largeur, hauteur, _ = image.shape
            
            # Ajouter une légende avec le numéro de sous-image
            if (displayLegend):
                numero_sous_image = fichier.split(".")[0].split("_")[1] + ' ' + fichier.split(".")[0].split("_")[2]  # Récupérer le numéro de sous-image à partir du nom de fichier
                axe.text(largeur-100, hauteur-50, "N° " + numero_sous_image, color='white', fontsize=12, ha='right', va='bottom', bbox=dict(facecolor='black', alpha=0.5)) # + ' - Gauche ' + str(xmin) + ' Bas ' +  str(ymin) + ' Droite ' +  str(xmax) + ' Haut ' +  str(ymax)
                axe.axis('off')
            
            ## Enregistrer l'image avec la légende
            #chemin_image_legende = os.path.join(path, f"legende_{numero_sous_image}.jpg")
            #plt.savefig(chemin_image_legende, bbox_inches='tight')
            
    # Fermer la figure
    #plt.close(fig)

#### Fonction pour récupérer tous les fichiers .shp d'un répertoire

In [ ]:
def get_shp_files(directory):
    """
    Récupère tous les fichiers .shp d'un répertoire de manière récursive.

    Args:
        directory (str): Le chemin du répertoire.

    Returns:
        pd.DataFrame: Un dataframe pandas avec trois colonnes : 'Nom du fichier', 'Chemin du fichier' et 'Contenu du fichier'.
    """
    # Créer un dataframe vide pour stocker les informations des fichiers .shp
    shp_files_df = pd.DataFrame(columns=['Répertoire','Nom du fichier', 'Chemin du fichier', 'Contenu du fichier'])

    # Parcourir le répertoire de manière récursive
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Vérifier si le fichier a l'extension .shp
            if file.endswith(".shp"):
                # Construire le chemin complet du fichier
                file_path = os.path.join(root, file)

                # Lire le contenu du fichier shp avec geopandas
                gdf = gpd.read_file(file_path)

                # Extraire le nom du répertoire
                dir_name = os.path.basename(os.path.dirname(file_path)).split('/')[-1]

                # Extraire le nom du fichier sans l'extension
                file_name = os.path.splitext(file)[0]

                # Créer un dataframe temporaire pour le fichier shp
                temp_df = pd.DataFrame({'Répertoire': [dir_name],
                                        'Nom du fichier': [file_name],
                                        'Chemin du fichier': [file_path],
                                        'Contenu du fichier': [gdf]})

                # Concaténer le dataframe temporaire avec le dataframe principal
                shp_files_df = pd.concat([shp_files_df, temp_df], ignore_index=True)

    return shp_files_df

#### Fonction pour afficher une image avec ses shapefile associés (ne marche pas !!!)

In [ ]:
def display_interactiveMap(img_path, shp_path):
    """
    Superpose un fichier shp sur une image jp2.

    Args:
        img_path (str): Le chemin du fichier jp2.
        shp_path (str): Le chemin du fichier shp.

    Returns:
        None
    """
    # Ouvrir le fichier jp2 avec rasterio
    with rasterio.open(img_path) as src:
        # Lire l'image jp2
        jp2_image = src.read()

        # Ouvrir le fichier shp avec geopandas
        gdf = gpd.read_file(shp_path)

        # Afficher l'image jp2
        plt.figure(figsize=(10, 10))
        rasterio.plot.show(jp2_image, transform=src.transform)

        # Superposer le fichier shp sur l'image jp2
        gdf.plot(ax=plt.gca(), facecolor='none', edgecolor='red')

        # Afficher la légende
        plt.legend(['Fichier shp'])

        # Afficher le graphique interactif
        plt.show()

#### Fonction pour charger l'image en définissant xrange et yrange a l'aide des bounds

In [ ]:
def load_image(path):
    with rasterio.open(path) as src:
        red_band = src.read(1).astype(np.float32)
        green_band = src.read(2).astype(np.float32)
        blue_band = src.read(3).astype(np.float32)
        resolution = src.res
        bounds = src.bounds

    # Normalize the bands
    normalized_red_band = normalize_band(red_band)
    normalized_green_band = normalize_band(green_band)
    normalized_blue_band = normalize_band(blue_band)

    # Stack the bands together
    rgb_image = np.stack((normalized_red_band, normalized_green_band, normalized_blue_band), axis=-1)

    coords = [(bounds.left, bounds.bottom), (bounds.right, bounds.top)]
    transform = rasterio.transform.from_bounds(*coords[0], *coords[1], len(rgb_image[0]), len(rgb_image))
    xrange = [coords[0][0], coords[1][0]]
    yrange = [coords[1][1], coords[0][1]]

    return rgb_image, xrange, yrange



#### Fonction de chargement des fichier shp en les catégorisant àl'aide des noms de dossiers

In [ ]:
# Define the load_shp function with an input for the file path
def load_shp(path):
    # Load data using GeoPandas
    bdtopo = gpd.read_file(path)

    # Add Category and Type columns
    category = path.split('/')[-2] 
    file_info = path.split('/')[-1][:-4]

    bdtopo.insert(0, 'Category', category)
    bdtopo.insert(1, 'Type', file_info)

    return bdtopo


#### Fonction de selection de centroides présent dans les coordonnees de l'image

In [ ]:
# Define a function to check if a polygon is within the specified x and y coordinate ranges

def isInMap(xrange, yrange):
    def my_function(polynom):
        x, y = polynom.centroid.x, polynom.centroid.y
        if xrange[0] < x and xrange[1] > x and yrange[0] < y and yrange[1] > y:
            return True
        else:
            return False

    return my_function

#### Fonction de traduction des coordonnées des centroides

In [ ]:
# Define a function to convert the coordinates of a polygon's centroid to a new coordinate system
def convert_centroid(map_size, xrange, yrange):
    def my_function(polygon):
        x, y = polygon.centroid.x, polygon.centroid.y
        x_new = (x - xrange[0]) / (xrange[1] - xrange[0]) * map_size[0]
        y_new = (y - yrange[0]) / (yrange[1] - yrange[0]) * map_size[1]
        return [x_new, y_new]

    return my_function


#### Fonction de traaduction des coordonnées des formes géométriques

In [ ]:
# Define a function to convert the coordinates of a polygon (or others) to a new coordinate system
def convert_polygon(map_size, xrange, yrange):
    def my_function(polygon):
        if polygon.wkt.lower()[:7] == "polygon":
            x, y = polygon.exterior.coords.xy
            x = x.tolist()
            y = y.tolist()
        elif polygon.wkt[:10].lower() == "linestring":
            x, y = polygon.coords.xy
            x = x.tolist()
            x += x[::-1]
            y = y.tolist()
            y += y[::-1]
        else:
            x = [1, 2]
            y = [1, 2]
        x = np.array(x)
        y = np.array(y)
        
        x_new = (x - xrange[0]) / (xrange[1] - xrange[0]) * map_size[0]
        y_new = (y - yrange[0]) / (yrange[1] - yrange[0]) * map_size[1]
        return [x_new, y_new]

    return my_function


#### Fonction pour transformer les coordonnées d'un polygone en liste de coordonnées

In [ ]:
# Define a function to generate coordinates for all polygons in a GeoDataFrame
def generate_x_polygons(xdata) :
    list_x = []
    for xpoly in xdata:
        list_x.extend(xpoly.tolist() + [None])
    list_x = list_x[:-1]
    return list_x


## 1) Récupération, compréhension, visualisation des données.

### Affichage de l'image originale

In [ ]:
# Chemin de l'image
imgPath = "./ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D02A_2019-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2020-07-00225/OHR_RVB_0M20_JP2-E080_LAMB93_D2A-2019/2A-2019-1160-6135-LA93-0M20-E080.jp2"

# Affichage de l'image
imageNormalized = displayImg(imgPath)

plt.figure(figsize=(15, 15))
plt.imshow(imageNormalized)
plt.show()

### Fichiers Shape

#### Visualisation des fichiers shp

In [ ]:
shpPath = "./Cadastre/BDTOPO_3-0_TOUSTHEMES_SHP_LAMB93_D02A_2019-03-15/"
df = get_shp_files(shpPath)

# Affichage propre
duplicates = df['Répertoire'].duplicated()
df.loc[duplicates, 'Répertoire'] = ''
df


#### Affichage des fichiers shapefile

In [ ]:
shp = gpd.read_file('/Users/raphael/Desktop/local.nosync/DétectionMI/BDTOPO_3-3_TOUSTHEMES_SHP_LAMB93_D02A_2022-12-15/BDTOPO/1_DONNEES_LIVRAISON_2022-12-00159/BDT_3-3_SHP_LAMB93_D02A-ED2022-12-15/BATI/BATIMENT.shp')

In [ ]:
shp.plot(figsize=(8,12))

In [ ]:
fig, ax = plt.subplots(figsize=(20,25))

shp.plot(ax=ax, color='black')

ax.set_xlim(1190000, 1195000)
ax.set_ylim(6080000, 6082500)
ax.set_xlabel('x axis')
ax.set_ylabel('y axis')

plt.show()

### Carte interactive

In [ ]:
# Image path
paths = "./ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D02A_2019-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2020-07-00225/OHR_RVB_0M20_JP2-E080_LAMB93_D2A-2019/2A-2019-1160-6135-LA93-0M20-E080.jp2"

# Load the image and its bounds
rgb_image, xrange, yrange = load_image(paths)



In [ ]:
# Find shapefiles in the specified folder
paths_shp = glob.glob(r'./BDTOPO_3-3_TOUSTHEMES_SHP_LAMB93_D02A_2022-12-15/BDTOPO/1_DONNEES_LIVRAISON_2022-12-00159/BDT_3-3_SHP_LAMB93_D02A-ED2022-12-15/BATI/*.shp')

# Load first shapefile
bdtopo = load_shp(paths_shp[0])

# Load the rest of the shapefiles and append them to the main GeoDataFrame
for path in paths_shp[1:]:
    bdtopo = pd.concat([bdtopo, load_shp(path)])

# Reset the index and drop the old index
bdtopo.reset_index(drop=True, inplace=True)

# Display the resulting GeoDataFrame
print(bdtopo)

In [ ]:
# Group the GeoDataFrame by 'Category' and 'Type' columns, and get the first geometry for each group
grouped_bdtopo = bdtopo.groupby(['Category', 'Type']).first()['geometry']

# Iterate through the available geometries and print their representations
for idx, geometry in grouped_bdtopo.items():
    print(f"Index: {idx}, Geometry: {repr(geometry)}")




In [ ]:
# Define the map size and coordinate ranges
map_size = [5000, 5000]

# Replace 'bdtopo', 'xrange', and 'yrange' with the actual GeoDataFrame and coordinate ranges you have
# Filter the GeoDataFrame to include only polygons within the specified coordinate ranges
bdtopo_zone = bdtopo[bdtopo['geometry'].apply(isInMap(xrange, [yrange[1], yrange[0]]))].copy()
print('Before:', bdtopo.shape, 'After:', bdtopo_zone.shape)

# Convert the coordinates of the centroid and polygon to the new coordinate system
bdtopo_zone['centroid'] = bdtopo_zone['geometry'].apply(convert_centroid(map_size, xrange, yrange))
# Get the x and y coordinates of the centroid and store them in separate columns
bdtopo_zone['xcentroid'] = bdtopo_zone['centroid'].apply(lambda x: x[0])
bdtopo_zone['ycentroid'] = bdtopo_zone['centroid'].apply(lambda x: x[1])

# Filter the GeoDataFrame to include desired forms within the specified coordinate ranges
bdtopo_point = bdtopo_zone[bdtopo_zone['geometry'].apply(lambda x: x.wkt.lower()[:5] == "point")]
bdtopo_zone = bdtopo_zone[bdtopo_zone['geometry'].apply(lambda x: x.wkt.lower()[:7] == "polygon" or x.wkt[:10].lower() == "linestring")]

# Convert the coordinates of the polygons and linestrings to the new coordinate system
bdtopo_zone['polygon'] = bdtopo_zone['geometry'].apply(convert_polygon(map_size, xrange, yrange))

# Get the x and y coordinates of the polygons and linestrings and store them in separate columns
bdtopo_zone['xpolygon'] = bdtopo_zone['polygon'].apply(lambda x: x[0])
bdtopo_zone['ypolygon'] = bdtopo_zone['polygon'].apply(lambda x: x[1])

bdtopo_zone.head()


In [ ]:
# Group the polygons and linestrings by their 'Type' column and aggregate their x and y coordinates into lists
bdtopo_zone_agregate = bdtopo_zone.groupby('Type').agg({'xpolygon' :list, 'ypolygon':list})

# Apply the 'generate_x_polygons' function to the coordinates columns to convert the x coordinates to a list format
bdtopo_zone_agregate['xpolygon_ready'] = bdtopo_zone_agregate['xpolygon'].apply(generate_x_polygons)
bdtopo_zone_agregate['ypolygon_ready'] = bdtopo_zone_agregate['ypolygon'].apply(generate_x_polygons)

bdtopo_zone_agregate

In [ ]:
# Group the points by their 'Type' column and aggregate their x and y centroid coordinates into lists
bdtopo_point_agregate = bdtopo_zone.groupby('Type').agg({'xcentroid':list, 'ycentroid' :list})
bdtopo_point_agregate

In [ ]:
# Define the show_map function with inputs for the image, scatter polygons, point markers, and their names
def show_map(rgb_image, scatters_data, scatters_list_name, points_data, points_list_name):
    # Create the base image
    image = px.imshow(cv2.resize(rgb_image, (5000,5000)))
    points = []

    # Add scatter polygons
    for i, (list_x, list_y) in enumerate(scatters_data):
        scatter_poly = go.Scatter(
            x=list_x,
            y=list_y,
            fill="toself",
            name=scatters_list_name[i]
        )
        points.append(scatter_poly)

    # Add point markers
    for i, (list_x, list_y) in enumerate(points_data):
        point_marker = go.Scatter(
            x=list_x,
            y=list_y,
            mode='markers',
            marker=dict(size=5),
            name=points_list_name[i]
        )
        points.append(point_marker)

    # Create the figure
    fig = go.Figure(data=[image.data[0]] + points)
    fig.update_xaxes(range=[0, 5000])
    fig.update_yaxes(range=[5000, 0])
    fig.update_layout(autosize=False, width=1000, height=800)

    fig.show()

# Call the show_map function with corrected inputs
show_map(rgb_image, bdtopo_zone_agregate[['xpolygon_ready', 'ypolygon_ready']].values, bdtopo_zone_agregate.index, bdtopo_point_agregate[['xcentroid', 'ycentroid']].values, bdtopo_point_agregate.index)


## 2) Transformation des images sous formes de tuiles, avec le masque des bâtiments.

### Découpe

#### Découpe de l'image

In [ ]:
# Répertoire source contenant toutes les mimages satellites
imgPath = "./Satellite/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D02A_2019-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2020-07-00225/OHR_RVB_0M20_JP2-E080_LAMB93_D2A-2019/2A-2019-1160-6135-LA93-0M20-E080.jp2"

# Nombre de pixels cible de chaque image découpée
nbPxX = 250
nbPxY = 250

# Répertoire de destination des images découpées
destPath = "./Decoupe_" + str(nbPxX) + "_" + str(nbPxY) + "/"

# Vide le répertoire de destination
emptyPath(destPath)

# Découpe de l'image
splitImg(nbPxX, nbPxY, imgPath, destPath)

#### Affichage de l'image découpée 

In [ ]:
srcPath = "./Decoupe_250_250/"
nbCol = 100
nbLines = 100
displayLegend = False

displayMultipleImg(srcPath, nbCol, nbLines, displayLegend)